In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np

In [ ]:
url = 'https://www.howardcountymd.gov/police/newsroom'
soup = BeautifulSoup(requests.get(url).text, "html.parser")

In [ ]:
def get_parced_data(raw):
    if raw.find_all("div", {"id": "daily-crime-bulletin"}):
        if raw.find_all("div", {"class": "paragraph--accordion__title"}):
            if raw.find_all("div", {"class": "field__item"}):
                return raw.find_all("div", {"class": "field__item"})

In [ ]:
parced_data = get_parced_data(soup)
crimes = [] 

for child in parced_data:
    if(child.find('h2')):
        crimes.append(child)

In [ ]:
# crimes[0]

In [ ]:
crimes_by_date = pd.DataFrame(columns=['publish_date','category','raw_crimes'])
for x in crimes:
    publish_date = x.find("h2").text.split(',',1)[1]
    raw_crimes = x.find_all("p")
    for c in raw_crimes:
        if c.find("u"):
            for cat in c.find_all("u"):
                category = 'cat.text'
                crimes_by_date.loc[len(crimes_by_date.index)] = [publish_date, category, raw_crimes] 

In [ ]:
# regex 
cat_regex = re.compile(r'<p><strong><u>(?P<cat>[a-zA-Z \/-]*)<\/u><br\/>')
city_regex = re.compile(r'(?P<city>[a-zA-z ]*),\s\d{5}')
zip_regex = re.compile(r'[a-zA-z ]*,\s(?P<zip>\d{5}):')
date_regex = re.compile(r'(?P<date>\w+\.?\s\d+-?\d*)')
time_regex = re.compile(r'\s(?P<time>\d{1,2}:\d{1,2}\s*[a.mp]*)')
street_regex = re.compile(r':\s?<\/strong>(?P<street>.*),\s?(March|Feb.|February)?')
notes_time_regex = re.compile(r':\d{1,2}\s*[a.mp]*<br\/>\s(.*)<')
notes_date_regex = re.compile(r'\w+\.?\s\d+-?\d*.*,\s(.*)<')

In [ ]:
by_crime_df = pd.DataFrame(columns=['publish_date', 'category', 'city', 'zip_code', 'street', 'crime_date', 'crime_time', 'add_notes'])

def by_crime(x):
    for c in x['raw_crimes']:
        cat = np.NaN
        street = np.NaN
        crime_date = np.NaN
        crime_time = np.NaN
        add_notes = np.NaN
        
        if cat_regex.findall(str(c)):
            cat = cat_regex.findall(str(c))[0]
        if city_regex.findall(str(c)):
            city = city_regex.findall(str(c))[0]
        if zip_regex.findall(str(c)):
            zip_code = zip_regex.findall(str(c))[0]
        # Fetch Street
        if street_regex.findall(str(c)):
            street = street_regex.findall(str(c))[0][0]
        if 'March' in street:
            street = street.split('March')[0]
        if 'Feb.' in street:
            street = street.split('Feb.')[0]
        if 'Feburary' in street:
            street = street.split('Feburary')[0]
        if date_regex.findall(str(c)):
            crime_date = date_regex.findall(str(c))[0]
        if time_regex.findall(str(c)):
            crime_time = time_regex.findall(str(c))[0]
        if notes_time_regex.findall(str(c)):
            add_notes = notes_time_regex.findall(str(c))[0]
#         if add_notes.isnull():
#             add_notes = notes_date_regex.findall(str(c))[0]
        by_crime_df.loc[len(by_crime_df.index)] = [x['publish_date'], cat, city, zip_code, street, crime_date, crime_time, add_notes]                    

In [ ]:
crimes_by_date.apply(by_crime,axis = 1)
by_crime_df['category'].fillna(method='ffill',inplace = True)
by_crime_df.drop_duplicates(inplace = True)
by_crime_df.reset_index(drop=True)

In [ ]:
by_crime_df['add_notes'].isnull().sum()

In [ ]:
by_crime_df.to_excel('data/howard-daily-crime-bulletin.xlsx', index=False)